In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score
import findspark
findspark.init()
from pyspark.sql import SparkSession
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

/var/folders/_l/b2ndqf2j6h5gcz0gbj9zfl4r0000gn/T/ipykernel_70622/4039071628.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.appName("Read CSV").getOrCreate()


24/03/07 18:39:28 WARN Utils: Your hostname, Aarons-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
24/03/07 18:39:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 18:39:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path = "Credit_Score_Data/credit_risk.csv"


In [4]:
df = spark.read.csv(file_path, header=True, inferSchema=True, sep=",")

df.show()

+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+
|        22|        59000|                 RENT|            123.0|         PERSONAL|         D|    35000|        16.02|          1|               0.59|                        Y|                         3|
|        21|         9600|                  OWN|              5.0|        EDUCATION|         B|     1000|        11.14|          0|                0.1|                        N|   

In [5]:
credit_df= df.toPandas()

credit_df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [6]:
spark.stop()

In [7]:
credit_df.dropna(inplace=True)

In [8]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28638 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  28638 non-null  int32  
 1   person_income               28638 non-null  int32  
 2   person_home_ownership       28638 non-null  object 
 3   person_emp_length           28638 non-null  float64
 4   loan_intent                 28638 non-null  object 
 5   loan_grade                  28638 non-null  object 
 6   loan_amnt                   28638 non-null  int32  
 7   loan_int_rate               28638 non-null  float64
 8   loan_status                 28638 non-null  int32  
 9   loan_percent_income         28638 non-null  float64
 10  cb_person_default_on_file   28638 non-null  object 
 11  cb_person_cred_hist_length  28638 non-null  int32  
dtypes: float64(3), int32(5), object(4)
memory usage: 2.3+ MB


In [9]:
non_numeric = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

In [10]:
df_credit = credit_df.drop(non_numeric, axis =1)

In [11]:
df_prep = df_credit.drop("loan_status", axis = 1)

In [12]:
X = df_prep
y = credit_df["loan_status"]


In [13]:
df_credit= pd.get_dummies(df_credit)

In [14]:
X= pd.concat([X, df_credit], axis=1)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
from keras import regularizers
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(1, activation='linear'))

In [18]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [19]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=25, batch_size=50, validation_split=0.2, verbose=1)


Epoch 1/25
344/344 [==============================] - 1s 1ms/step - loss: 0.6164 - accuracy: 0.9543 - val_loss: 9.8941e-05 - val_accuracy: 1.0000
Epoch 2/25
344/344 [==============================] - 0s 899us/step - loss: 0.0011 - accuracy: 0.9999 - val_loss: 1.4555e-05 - val_accuracy: 1.0000
Epoch 3/25
344/344 [==============================] - 0s 800us/step - loss: 9.0999e-04 - accuracy: 0.9999 - val_loss: 2.6015e-05 - val_accuracy: 1.0000
Epoch 4/25
344/344 [==============================] - 0s 799us/step - loss: 9.3183e-04 - accuracy: 0.9999 - val_loss: 6.6890e-05 - val_accuracy: 1.0000
Epoch 5/25
344/344 [==============================] - 0s 813us/step - loss: 9.1310e-04 - accuracy: 0.9999 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/25
344/344 [==============================] - 0s 857us/step - loss: 9.3158e-04 - accuracy: 0.9999 - val_loss: 4.8412e-07 - val_accuracy: 1.0000
Epoch 7/25
344/344 [==============================] - 0s 798us/step - loss: 8.9774e-04 - accuracy:

In [20]:
# Evaluate the model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test)
print("Accuracy = ", model_accuracy)

224/224 [==============================] - 0s 460us/step - loss: 0.0000e+00 - accuracy: 1.0000
Accuracy =  1.0


In [21]:
# Convert X_test to float
X_test = X_test.astype(float)

In [22]:
# Make predictions
y_pred = model.predict(X_test)

224/224 [==============================] - 0s 476us/step


In [25]:
categorical_predictions = np.argmax(y_pred, axis=1)

In [26]:
accuracy = accuracy_score(y_test, categorical_predictions)

In [27]:
print(accuracy)

0.7777932960893855
